In [1]:
from ibots_db.schema import Data
from pydantic import BaseModel
from pathlib import Path
import shutil

In [2]:
Data.model_fields

{'people': FieldInfo(annotation=ForwardRef('dict[str, Person]'), required=True),
 'orgs': FieldInfo(annotation=ForwardRef('Orgs'), required=True),
 'skills': FieldInfo(annotation=ForwardRef('dict[str, Skill]'), required=True),
 'technologies': FieldInfo(annotation=ForwardRef('dict[str, Technology]'), required=True),
 'images': FieldInfo(annotation=dict[str, Annotated[Path, PathType]], required=True)}

In [3]:
Data.model_rebuild()

True

In [5]:
Data.model_fields

{'people': FieldInfo(annotation=dict[str, Person], required=True),
 'orgs': FieldInfo(annotation=Orgs, required=True),
 'skills': FieldInfo(annotation=dict[str, Skill], required=True),
 'technologies': FieldInfo(annotation=dict[str, Technology], required=True),
 'images': FieldInfo(annotation=dict[str, Annotated[Path, PathType]], required=True)}

In [6]:
tech = Data.model_fields['technologies']

In [7]:
tech.annotation

dict[str, ibots_db.schema.Technology]

In [8]:
tech.get_default()

PydanticUndefined

In [9]:
tech.annotation

dict[str, ibots_db.schema.Technology]

In [10]:
from typing import get_args, get_origin

In [11]:
get_args(tech.annotation)

(str, ibots_db.schema.Technology)

In [12]:
get_origin(tech.annotation)

dict

In [13]:
get_origin(tech.annotation)

dict

---

In [14]:
aa = {'a': 3}


In [83]:
from tempfile import TemporaryDirectory
from ibots_db.funs import DATA_PATH
from ibots_db.schema import Technology
import yaml

# assert key is an attribute of Data schema
key = 'technologies'
data = {
    'tech1': Technology(name='C-Sharp', icon='icon c-sharp', homepage='https://www.c-sharp.com'),
    'tech2': Technology(name='C++', icon='icon c++', homepage='https://www.cpp.com')
    }

field = Data.model_fields[key]

# check that BaseModel given is the same type specified by the schema
assert get_origin(field.annotation) is dict
assert isinstance(data, dict)
key_type, value_type = get_args(field.annotation)
assert key_type is str
assert issubclass(value_type, BaseModel)
assert all(type(value) is value_type for key, value in data.items())

# Make yaml out of the data
yaml_data = {k + '.yaml': yaml.dump(value.model_dump(mode='json'), Dumper=yaml.Dumper) for k, value in data.items()}
for val in yaml_data.values():
    print(val)

# make a temporary directory and copy the db files into it.
temp_dir = Path(TemporaryDirectory(prefix='dbcheck').name)
print(temp_dir)
shutil.copytree(DATA_PATH, temp_dir, dirs_exist_ok=True)

if not temp_dir.joinpath(key).with_suffix('.yaml').exists():
    for k,value in yaml_data.items():
        file_path = temp_dir.joinpath(key).joinpath(k)
        file_path.parent.mkdir(parents=True, exist_ok=True)
        file_path.write_text(value)

else: 
    temp_txt = temp_dir.joinpath(key).with_suffix('.yaml')
    parsed_data = {key: yaml.safe_load(value) for key, value in yaml_data.items()}
    with open(temp_txt, 'w') as file:
        yaml.dump(parsed_data, file, default_flow_style=False)

print(yaml_data)
print(temp_dir.joinpath(key).with_suffix('.txt'))

# try it in real
print(DATA_PATH)

homepage: https://www.c-sharp.com/
icon: icon c-sharp
name: C-Sharp

homepage: https://www.cpp.com/
icon: icon c++
name: C++

C:\Users\SANGEE~1\AppData\Local\Temp\dbcheck_2wrkx_s
{'tech1.yaml': 'homepage: https://www.c-sharp.com/\nicon: icon c-sharp\nname: C-Sharp\n', 'tech2.yaml': 'homepage: https://www.cpp.com/\nicon: icon c++\nname: C++\n'}
C:\Users\SANGEE~1\AppData\Local\Temp\dbcheck_2wrkx_s\technologies.txt
C:\Users\sangeetha\OurWebsite\group_data\data


---

In [ ]:
issubclass(key_type, BaseModel)

In [ ]:
BaseModel.model_dump?

In [ ]:
BaseModel in value_type.mro()

In [ ]:
value_type.mro()

In [ ]:
class A: ...
class B(A): ...
class E(A): ...
class D(E): ...
class C(D, B): ...


In [ ]:
A.mro()

In [ ]:
B.mro()

In [ ]:
D.mro()

In [ ]:
C.mro()

In [ ]:
type(C.mro())